建立一個torch model給hls4ml轉換

一開始不會有model4hls.pth，這個是從baseline model的transformer encoder取出來的weight

In [3]:
# from torch import nn
# import torch
# class Transformer4HLS(torch.nn.Module):
#       def __init__(self, d_model, nhead, num_encoder_layers, dim_feedforward, dropout, activation, norm_first, device):
#           super().__init__()
#           self.d_model = d_model
#           self.nhead = nhead
#           self.num_encoder_layers = num_encoder_layers
#           self.dim_feedforward = dim_feedforward
#           self.dropout = dropout
#           self.activation = activation
#           self.norm_first = norm_first
#           self.device = device
#           self._init_transformer()

#       def _init_transformer(self):
#           norm = nn.LayerNorm(self.d_model)
#           self.transformer_encoder = nn.TransformerEncoder(
#                                         nn.TransformerEncoderLayer(d_model=self.d_model, 
#                                                                    nhead=self.nhead,
#                                                                    dim_feedforward=self.dim_feedforward,
#                                                                    dropout=self.dropout,
#                                                                    activation=self.activation,
#                                                                    norm_first=self.norm_first,
#                                                                    device=self.device),
#                                         self.num_encoder_layers,
#                                         norm=norm
#                                       )

#       def forward(self, src):  
#           output = self.transformer_encoder(src)
#           return output
# model4hls = torch.load('./model4hls.pth', map_location=torch.device('cpu'))
# print(model4hls)

In [4]:
# from torch.fx import symbolic_trace

# traced_model = symbolic_trace(model4hls)

In [5]:
# for node in traced_model.graph.nodes:
#     print(node)

In [6]:
# print(traced_model.graph)

In [7]:
import os
import random
import argparse
import json
from warnings import warn
from typing import List, Dict
from pathlib import Path
from functools import partial
from textwrap import wrap
from contextlib import suppress
from statistics import mean, stdev

import numpy as np
from tqdm import tqdm
import wandb
import matplotlib.pyplot as plt
from mpl_toolkits import axes_grid1
from einops import rearrange, reduce
from timm.models import create_model
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import models
from datasets import build_dataset
from train import get_args_parser, adjust_config, set_seed, set_run_name, count_params

In [8]:
parser = argparse.ArgumentParser('DeiT training and evaluation script', parents=[get_args_parser()])
parser.add_argument('--compute_attention_average', action='store_true')
parser.add_argument('--compute_attention_cka', action='store_true')
parser.set_defaults(output_dir='results_inference')
args = parser.parse_args(args=[])
adjust_config(args)

args.model = 'evit_deit_tiny_patch16_224.fb_in1k'
args.dataset_name = 'cotton'
args.dataset_root_path = '../../data/cotton'
args.folder_train = 'cotton_square_new'
args.folder_val = 'cotton_square_new'
args.folder_test = 'cotton_square_new'
args.device = 'cpu'
# args.keep_rate = [0.1]
# args.finetune = './results_clca/cotton_evit_vit_base_patch16_224.orig_in21k_0.1_30.pth'
# args.finetune = './results_clca/cotton_evit_vit_base_patch16_224.orig_in21k_0.1_cla_clc_30.pth'
args.finetune = None
args.input_size = 16
args.model_depth = 12

{'dataset_name': 'soylocal', 'dataset_root_path': '../../data/soylocal', 'df_train': 'train.csv', 'df_trainval': 'train_val.csv', 'df_val': 'val.csv', 'df_test': 'test.csv', 'folder_train': 'soybean200_square', 'folder_val': 'soybean200_square', 'folder_test': 'soybean200_square', 'df_classid_classname': False}
{'pretrained': True}
{'horizontal_flip': True}


In [9]:
set_seed(args.seed)

dataset_train, args.num_classes = build_dataset(is_train=True, args=args)
dataset_val, _ = build_dataset(is_train=False, args=args)

sampler_train = torch.utils.data.RandomSampler(dataset_train)
sampler_val = torch.utils.data.SequentialSampler(dataset_val)

train_loader = torch.utils.data.DataLoader(
    dataset_train, sampler=sampler_train,
    batch_size=args.batch_size,
    num_workers=args.num_workers,
    pin_memory=args.pin_mem,
    drop_last=False,
)
test_loader = torch.utils.data.DataLoader(
    dataset_val, sampler=sampler_val,
    batch_size=args.batch_size,
    num_workers=args.num_workers,
    pin_memory=args.pin_mem,
    drop_last=False
)

if args.finetune and args.ifa_head and args.clc:
    args.setting = 'ft_clca'
elif args.finetune and args.ifa_head:
    args.setting = 'ft_cla'
elif args.finetune:
    args.setting = 'ft_bl'
else:
    args.setting = 'fz_bl'

Compose(
    Resize(size=(18, 18), interpolation=bicubic, max_size=None, antialias=True)
    RandomCrop(size=(16, 16), padding=None)
    RandomHorizontalFlip(p=0.5)
    ToTensor()
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
)
Compose(
    Resize(size=(18, 18), interpolation=bicubic, max_size=None, antialias=True)
    CenterCrop(size=(16, 16))
    ToTensor()
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
)


In [10]:
print(f"Creating model: {args.model}")
model = create_model(
    args.model,
    pretrained=True,
    pretrained_cfg=None,
    pretrained_cfg_overlay=None,
    num_classes=1000,
    drop_rate=args.drop,
    drop_path_rate=args.drop_path,
    drop_block_rate=None,
    img_size=args.input_size,
    args = args
)
if args.dataset_name.lower() != "imagenet":
    model.reset_classifier(args.num_classes)
print(model)

model.to(args.device)

model.eval()

if args.finetune:
    checkpoint = torch.load(args.finetune, map_location='cpu')
    model.load_state_dict(checkpoint['model'], strict=True)

Creating model: evit_deit_tiny_patch16_224.fb_in1k
[] []
EViT(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 192, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=192, out_features=576, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (pre_softmax): Identity()
        (proj): Linear(in_features=192, out_features=192, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=192, out_features=768, bias=True)
        (act): GELU(approximate='non

In [11]:
print('Total parameters (M): ', count_params(model) / (1e6))
print('Trainable parameters (M): ', count_params(model, trainable=True) / (1e6))

Total parameters (M):  5.502416
Trainable parameters (M):  5.502416


### 建立`model4hls`使得HLS4ML可以識別，並轉移DeiT-T `model`權重至`model4hls`

In [12]:
class Transformer4HLS(torch.nn.Module):
      def __init__(self, d_model, nhead, num_encoder_layers, dim_feedforward, dropout, activation, norm_first, device):
          super().__init__()
          self.d_model = d_model
          self.nhead = nhead
          self.num_encoder_layers = num_encoder_layers
          self.dim_feedforward = dim_feedforward
          self.dropout = dropout
          self.activation = activation
          self.norm_first = norm_first
          self.device = device
          self._init_transformer()

      def _init_transformer(self):
          norm = nn.LayerNorm(self.d_model)
          self.transformer_encoder = nn.TransformerEncoder(
                                        nn.TransformerEncoderLayer(d_model=self.d_model, 
                                                                   nhead=self.nhead,
                                                                   dim_feedforward=self.dim_feedforward,
                                                                   dropout=self.dropout,
                                                                   activation=self.activation,
                                                                   norm_first=self.norm_first,
                                                                   device=self.device),
                                        self.num_encoder_layers,
                                        norm=norm
                                      )

      def forward(self, src):  
          output = self.transformer_encoder(src)
          return output

torch.manual_seed(0)
model4hls = Transformer4HLS(d_model=192, 
                          nhead=3, 
                          num_encoder_layers=args.model_depth, 
                          dim_feedforward=768, 
                          dropout=0, 
                          activation='gelu', 
                          norm_first=True, 
                          device='cpu')

model4hls.eval()

for i in range(args.model_depth):
    model4hls.transformer_encoder.layers[i].self_attn.in_proj_weight    = model.blocks[i].attn.qkv.weight
    model4hls.transformer_encoder.layers[i].self_attn.in_proj_bias      = model.blocks[i].attn.qkv.bias
    model4hls.transformer_encoder.layers[i].self_attn.out_proj.weight   = model.blocks[i].attn.proj.weight
    model4hls.transformer_encoder.layers[i].self_attn.out_proj.bias     = model.blocks[i].attn.proj.bias
    model4hls.transformer_encoder.layers[i].linear1.weight              = model.blocks[i].mlp.fc1.weight
    model4hls.transformer_encoder.layers[i].linear1.bias                = model.blocks[i].mlp.fc1.bias
    model4hls.transformer_encoder.layers[i].linear2.weight              = model.blocks[i].mlp.fc2.weight
    model4hls.transformer_encoder.layers[i].linear2.bias                = model.blocks[i].mlp.fc2.bias
    model4hls.transformer_encoder.layers[i].norm1.weight                = model.blocks[i].norm1.weight
    model4hls.transformer_encoder.layers[i].norm1.bias                  = model.blocks[i].norm1.bias
    model4hls.transformer_encoder.layers[i].norm2.weight                = model.blocks[i].norm2.weight
    model4hls.transformer_encoder.layers[i].norm2.bias                  = model.blocks[i].norm2.bias
model4hls.transformer_encoder.norm.weight   = model.norm.weight
model4hls.transformer_encoder.norm.bias     = model.norm.bias

torch.save(model4hls, './model4hls_{}.pth'.format(args.input_size))

/home/chengwei/anaconda3/envs/auto_hls4ml/lib/python3.10/site-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


### 比較model和model4hls的輸出

In [13]:
for idx in range(1):
    random_tensor = torch.randn(1, 3, args.input_size, args.input_size)
    # print(random_tensor)
    with torch.no_grad():
        x = model.patch_embed(random_tensor)
        x = model._pos_embed(x)
        x = model.patch_drop(x)
        x = model.norm_pre(x)
        print('Input shape of encoders = {}'.format(x.shape))
        out = x
        out2 = x
        # out, left_token, sample_idx, compl = model.blocks[0](x)
        # out2 = model4hls.transformer_encoder.layers[0](x.permute(1, 0, 2))
        for i, blk in enumerate(model.blocks):
            out, left_token, sample_idx, compl = blk(out)
        out = model.norm(out)
        out2 = model4hls(out2.permute(1, 0, 2))
        out2 = out2.permute(1, 0, 2)
        # print(out.shape)
        # print(out2.shape)
        # print(out)
        # print(out2)
        difference = (out - out2).sum()
        print('Difference between pytorch model and model4hls = {}'.format(difference))

Input shape of encoders = torch.Size([1, 2, 192])
Difference between pytorch model and model4hls = -7.995311170816422e-05


### 生成`transformer_quant_config`量化transformer encoder，並calibrate quantized model觀察quantizer的數值範圍並重新生成calibated `transformer_quant_config`
#### Tips : 由於calibration可能會很久(取決於使用多大的calibation dataset)，建議將calibrated `transformer_quant_config`存檔

In [14]:
from quantizers import *
from synchronizer import *
import hls4ml
import json
import copy
from pprint import pprint

def load_transformer_quant_config(quant_config_path: str = "./quant_config.json",
                                  norm_quant_config_path: str = "./norm_quant_config.json",
                                  num_layers: int = 1) -> dict:
    with open(quant_config_path, 'r') as f:
        quant_config = json.load(f)
    with open(norm_quant_config_path, 'r') as f:
        norm_quant_config = json.load(f)
    transformer_quant_config = {}
    for i in range(num_layers):
        transformer_quant_config[i] = copy.deepcopy(quant_config)
    transformer_quant_config['norm'] = copy.deepcopy(norm_quant_config)
    return transformer_quant_config

In [15]:
transformer_quant_config = load_transformer_quant_config(num_layers=args.model_depth)
print('Before calibration:')
pprint(transformer_quant_config)
qmodel = QTransformerEncoder([QTransformerEncoderLayer(192, 
                                                       3, 
                                                       768, 
                                                       activation='gelu', 
                                                       quant_config=transformer_quant_config[i], 
                                                       calibration=True, 
                                                       device='cpu') for i in range(args.model_depth)], 
                             args.model_depth, 
                             QLayerNorm(192, quant_config=transformer_quant_config['norm'], calibration=True, device='cpu'),
                             TorchQuantizer(bitwidth=18, int_bitwidth=5, signed=True, calibration=True),
                             dtype=torch.float64)
qmodel.transfer_weights(model4hls)
qmodel.to(torch.device('cpu'))
qmodel.eval()

# # 校準
# print('Calibrating...')
# images, target = next(iter(test_loader))
# with torch.no_grad():
#     x = model.patch_embed(images[0:1])
#     x = model._pos_embed(x)
#     x = model.patch_drop(x)
#     x = model.norm_pre(x)
#     print(x.min())
#     print(x.max())
#     transformer_quant_config = calibrate_transformer(qmodel, transformer_quant_config, x.permute(1, 0, 2).type(torch.float64).to(torch.device('cpu')))

# print('After calibration:')
# pprint(transformer_quant_config)
# #save transformer_quant_config
# torch.save(transformer_quant_config, './transformer_quant_config_{}.pth'.format(args.input_size))

Before calibration:
{0: {'ffn': {'in_proj': {'bias': {'bitwidth': 18, 'int_bitwidth': 8},
                         'input': {'bitwidth': 18,
                                   'int_bitwidth': 8,
                                   'quantize': False},
                         'output': {'bitwidth': 18,
                                    'int_bitwidth': 8,
                                    'quantize': False},
                         'weight': {'bitwidth': 18, 'int_bitwidth': 8}},
             'out_proj': {'bias': {'bitwidth': 18, 'int_bitwidth': 8},
                          'input': {'bitwidth': 18, 'int_bitwidth': 8},
                          'output': {'bitwidth': 18, 'int_bitwidth': 8},
                          'weight': {'bitwidth': 18, 'int_bitwidth': 8}}},
     'input': {'bitwidth': 18, 'int_bitwidth': 8},
     'norm1': {'bias': {'bitwidth': 18, 'int_bitwidth': 8},
               'input': {'bitwidth': 18, 'int_bitwidth': 8},
               'mean': {'bitwidth': 18, 'int_bitwid

QTransformerEncoder(
  (layers): ModuleList(
    (0-11): 12 x QTransformerEncoderLayer(
      (self_attn): QFlashMultiheadAttention(
        (out_proj): QLinear(
          in_features=192, out_features=192, bias=True
          (weight_qtzr): TorchQuantizer()
          (bias_qtzr): TorchQuantizer()
          (input_qtzr): TorchQuantizer()
          (output_qtzr): TorchQuantizer()
        )
        (in_proj): QLinear(
          in_features=192, out_features=576, bias=True
          (weight_qtzr): TorchQuantizer()
          (bias_qtzr): TorchQuantizer()
          (input_qtzr): TorchQuantizer()
          (output_qtzr): TorchQuantizer()
        )
        (scale_qtzr): TorchQuantizer()
        (row_sum_qtzr): TorchQuantizer()
        (exp_input_qtzr): TorchQuantizer()
        (exp_output_qtzr): TorchQuantizer()
        (inv_input_qtzr): TorchQuantizer()
        (inv_output_qtzr): TorchQuantizer()
        (attn_out_qtzr): TorchQuantizer()
      )
      (linear1): Linear(in_features=192, out_f

In [16]:
#load transformer_quant_config
transformer_quant_config = torch.load('./transformer_quant_config_{}.pth'.format(args.input_size))
pprint(transformer_quant_config)

{0: {'ffn': {'in_proj': {'bias': {'bitwidth': 18, 'int_bitwidth': 3},
                         'input': {'bitwidth': 18,
                                   'int_bitwidth': -inf,
                                   'quantize': False},
                         'output': {'bitwidth': 18,
                                    'int_bitwidth': -inf,
                                    'quantize': False},
                         'weight': {'bitwidth': 18, 'int_bitwidth': 1}},
             'out_proj': {'bias': {'bitwidth': 18, 'int_bitwidth': 3},
                          'input': {'bitwidth': 18, 'int_bitwidth': 4},
                          'output': {'bitwidth': 18, 'int_bitwidth': 4},
                          'weight': {'bitwidth': 18, 'int_bitwidth': 0}}},
     'input': {'bitwidth': 18, 'int_bitwidth': 8},
     'norm1': {'bias': {'bitwidth': 18, 'int_bitwidth': 2},
               'input': {'bitwidth': 18, 'int_bitwidth': 5},
               'mean': {'bitwidth': 18, 'int_bitwidth': -3},
    

/tmp/ipykernel_8660/1672244013.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  transformer_quant_config = torch.load('./transformer_quant_config_{}.pth'.format(args.inpu

跑baseline model的準確度(跟model4hls比對)

In [17]:
# import utils
# from timm.utils import accuracy
# amp_autocast = suppress

# criterion = torch.nn.CrossEntropyLoss()

# metric_logger = utils.MetricLogger(delimiter="  ")
# header = 'Val:'

# # switch to evaluation mode
# model.eval()

# for images, target in metric_logger.log_every(test_loader, 10, header):
#     images = images.to(args.device, non_blocking=True)
#     target = target.to(args.device, non_blocking=True)

#     # compute output
#     with amp_autocast():
#         output = model(images)
#     if isinstance(output, (tuple, list)):
#         output = output[0]

#     loss = criterion(output, target)
#     acc1, acc5 = accuracy(output, target, topk=(1, 5))

#     batch_size = images.shape[0]
#     metric_logger.update(loss=loss.item())
#     metric_logger.meters['acc1'].update(acc1.item(), n=batch_size)
#     metric_logger.meters['acc5'].update(acc5.item(), n=batch_size)

# # gather the stats from all processes
# metric_logger.synchronize_between_processes()
# print('* Acc@1 {top1.global_avg:.3f} Acc@5 {top5.global_avg:.3f} loss {losses.global_avg:.3f}'
#         .format(top1=metric_logger.acc1, top5=metric_logger.acc5, losses=metric_logger.loss))

### 生成`state` for Simulated Annealing(若沒有要透過Simulated Annealing優化，這邊只是作為同步`quant_config`和`hls_config`的方法)並測試sync_quant_config
- `state`包含影響BRAM數目的變數`BRAMstate`以及不影響BRAM數目的變數`DSPstate`(或者說影響DSP數目的變數，但目前並沒有 ***TODO : 將DSP相關變數加入Design Search Space***)
- `num_layers`為Transformer Block的數量
- `weight_bits`主要包含MHSA的兩個linear的weight(或者是Q、K、V的weight以及O的weight)、FFN的兩個linear layer的weight的bit-wdith
- `table_input_bits`和`table_output_bits`包含，MHSA的exponential、倒數查表、LayerNorm的variance查表、FFN的GeLU(CDF)查表。
  - 2的`table_input_bits`次方即為Look-up table的Entry數量，因此這個數值只會設置約12上下(設成32可能會讓軟體本身overflow)
  - `table_output_bits`即為Look-up table的width。由於BRAM的配置18 bits或9 bits的使用效率最高，因此這邊通常只會是這兩個數值或其倍數
- `intermediate_bits`包含MHSA中的QKV cache，由於對Deit-tiny來說，QKV所需緩存很大，因此使用UltraRAM實現，而UltraRAM使用72 bits = 24 bits* 3 heads最有效率，並不將此列入BRAM計算(***TODO : KV cache存至HBM***)
- `result_bits`包含所有layer的output，使用FIFO實現，由於選取適當的Tile size可減小FIFO深度，所以使用LUTRAM實現並不列入BRAM計算(***TODO : Formulize FIFO深度與Tile size的關係以估計LUTRAM數量***)

In [18]:
BRAMstate = gen_init_BRAMaware_state(num_layers=args.model_depth, 
                                #    weight_bits=8, 
                                   weight_bits=18, 
                                   table_input_bits=12, 
                                   table_output_bits=18, 
                                   intermediate_bits=24, 
                                #    result_bits=18)
                                   result_bits=18)
DSPstate = gen_init_nonBRAMaware_state(num_layers=args.model_depth)
state = {**BRAMstate, **DSPstate}

config = hls4ml.utils.config_from_pytorch_model(model4hls, 
                                              granularity='name',
                                              backend='Vitis',
                                              input_shapes=[[1, (args.input_size/16)**2+1, 192]], 
                                              default_precision='ap_fixed<18,5,AP_RND_CONV,AP_SAT>', 
                                              inputs_channel_last=True, 
                                              transpose_outputs=False)

valid = sync_quant_config(transformer_quant_config, config, state)
print(valid)

True


### 建立quantize model `qmodel` 並載入calibared和sync up後的 `transformer_quant_config`。配置HLS config中的Tile size以最大化BRAM以及硬體使用效率並產生 `hls_model` 和HLS project

In [19]:
qmodel = QTransformerEncoder([QTransformerEncoderLayer(embed_dim=192, 
                                                       num_heads=3, 
                                                       hidden_dim=768, 
                                                       activation='gelu', 
                                                       quant_config=transformer_quant_config[i], 
                                                       calibration=False, 
                                                       device='cpu') for i in range(args.model_depth)], 
                             args.model_depth, 
                             QLayerNorm(normalized_shape=192, 
                                        quant_config=transformer_quant_config['norm'], 
                                        calibration=False, 
                                        device='cpu'),
                             TorchQuantizer(bitwidth=18, int_bitwidth=5, signed=True, calibration=False),
                             dtype=torch.float64)
qmodel.transfer_weights(model4hls)
qmodel.to(torch.device('cpu'))
qmodel.eval()
for layer_config in config['LayerName'].keys():
    if layer_config.endswith('self_attn'):
        config['LayerName'][layer_config]['TilingFactor'] = [1,1,1]
    elif layer_config.endswith('ffn'):
        config['LayerName'][layer_config]['TilingFactor'] = [1,1,12]
hls_model = hls4ml.converters.convert_from_pytorch_model(
                                                            model4hls,
                                                            [[1, int((args.input_size/16)**2+1), 192]],
                                                            output_dir='./hls/deit_tiny_w8_Bdk-1_Bffn-12_{}'.format(args.input_size),
                                                            project_name='myproject',
                                                            backend='Vitis',
                                                            part='xcu55c-fsvh2892-2L-e',
                                                            #board='alveo-u55c',
                                                            hls_config=config,
                                                            io_type='io_tile_stream',
                                                        )
hls_model.compile()

Writing HLS project


KeyboardInterrupt: 

### 比較`qmodel` 、 `hls_model`和`model4hls`的輸出。理論上，`qmodel` 和 `hls_model`的輸入要一致

In [ ]:
images, target = next(iter(test_loader))
with torch.no_grad():
    x = model.patch_embed(images[0:1])
    x = model._pos_embed(x)
    x = model.patch_drop(x)
    x = model.norm_pre(x)
    print('Double check input shape of encoders = {}'.format(x.shape))
    print(x.shape) 
    # print(x)
    np.savetxt('hls/deit_tiny_w8_Bdk-1_Bffn-12_{}/tb_data/tb_input_features.dat'.format(args.input_size), x.numpy().flatten().reshape(1, -1), fmt="%.6f", delimiter=" ")
    output = qmodel(x.permute(1, 0, 2).type(torch.float64))
    output = output.permute(1, 0, 2)
    encoder_out2 = model4hls(x.permute(1, 0, 2))
    encoder_out2 = encoder_out2.permute(1, 0, 2)
    hls_output = hls_model.predict(x.numpy())
    np.savetxt('hls/deit_tiny_w8_Bdk-1_Bffn-12_{}/tb_data/tb_output_predictions.dat'.format(args.input_size), encoder_out2.numpy().flatten().reshape(1, -1), fmt="%.6f", delimiter=" ")
    test_output1 = hls_output - encoder_out2.flatten().numpy()
    test_output1_sum = np.sum(test_output1)
    print('Sum of difference between qmodel output and hls_output = {}'.format((output.flatten().numpy() - hls_output).sum()))
    # print(test_output1)
    print('Sum of difference between pytorch output and hls_output = {}'.format(test_output1_sum))
    # print(output)
    # print(encoder_out2)
    # print(hls_output)

torch.Size([1, 101, 192])


Exception: Input size mismatch, got (), expected [192, 106, 1]

In [ ]:
print(x.shape)

torch.Size([1, 101, 192])


### 預測`hls_model`以及產生的HLS project的BRAM數目，這會`state`的配置與tile size的大小有關

In [ ]:
from estimators import *
def layer_estimater(quant_config):
    bram_dict = {}  
    for layer_name in quant_config.keys():
        bram_dict[layer_name] = {}
        for var_name in quant_config[layer_name].keys():
            #pprint(quant_config[layer_name][var_name])
            bram_dict[layer_name][var_name] = VivadoVariableBRAMEstimator(name=var_name,**quant_config[layer_name][var_name])

    num_ram = 0
    for layer_name in bram_dict.keys():
        for var_name in bram_dict[layer_name].keys():
            ram_est = bram_dict[layer_name][var_name]
            num_ram += ram_est.get_num_ram()
    return num_ram
num_ram = layer_estimater(parse_hls_model(hls_model))
print(num_ram)

9
9
9
9
9
9
9
9
9
9
9
9
3754.5
